In [1]:
from dask.distributed import Client
from dask.diagnostics import ProgressBar
import dask
import dask.delayed
import numpy as np
import xarray as xr
import geopandas as gpd

from matplotlib import pyplot as plt
from rasterio import RasterioIOError
from tqdm.autonotebook import tqdm
%matplotlib inline
import sklearn.linear_model
import skimage.morphology
import skimage.segmentation
import skimage.future
#import richdem as rd
import scipy.ndimage
import dask
import math
import scipy.sparse
import shapefile
import shapely
import rioxarray
import cartopy as crt
from shapely.geometry import mapping
#import networkx as nx
import matplotlib
import pandas as pd

#import cv2
import gcsfs
import json
import h5py



<ipython-input-1-b311f7ad264f>:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
cd  ..

/home/jovyan


In [3]:
import os
import dask
from dask_gateway import Gateway
from dask.distributed import Client
import dask_gateway
import dask.array as da

## Open Cluster

In [4]:
# get the dask-gateway version
dask_gateway.__version__

# show the default dask-gateway settings
dask.config.config['gateway']

# show the current default image to be started on workers
os.environ['JUPYTER_IMAGE_SPEC']

# use the same GDAL environment settings as we did for the single COG case
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

gateway = Gateway()
#options = gateway.cluster_options()
#options.environment = env 
#cluster = gateway.new_cluster(options)
#cluster.scale(4) # let's get the same number of "workers" as our previous LocalCluster examples
cluster = gateway.new_cluster()

gateway.list_clusters()

[ClusterReport<name=prod.c3caa6bb6cf34c9cbdacd075579634dd, status=RUNNING>]

In [5]:
gateway.list_clusters()

[ClusterReport<name=prod.c3caa6bb6cf34c9cbdacd075579634dd, status=RUNNING>]

In [6]:
# connect a client
# the distributed client is used for running parallel tasks with Dask
client = Client(cluster)

In [7]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.c3caa6bb6cf34c9cbdacd075579634dd Dashboard: /services/dask-gateway/clusters/prod.c3caa6bb6cf34c9cbdacd075579634dd/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [8]:
cd

/home/jovyan


In [9]:
from dask.distributed import Client
from dask.diagnostics import ProgressBar
import dask
import dask.delayed
import numpy as np
import xarray as xr
import geopandas as gpd

from matplotlib import pyplot as plt
from rasterio import RasterioIOError
from tqdm.autonotebook import tqdm
%matplotlib inline
import sklearn.linear_model
import skimage.morphology
import skimage.segmentation
import skimage.future
#import richdem as rd
import scipy.ndimage
import dask
import math
import scipy.sparse
import shapefile
import shapely
import rioxarray
from shapely.geometry import mapping
import networkx as nx
import matplotlib
import pandas as pd

#import cv2
import gcsfs
import json




In [10]:
ice_shelf = 'LarsenC'

In [11]:
cd

/home/jovyan


In [12]:
with open('ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
gcs = gcsfs.GCSFileSystem(token=token)
mapper = gcs.get_mapper(np.str('gs://ldeo-glaciology/REMA/processed/'+ice_shelf))

<ipython-input-12-5ffcf5648572>:4: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mapper = gcs.get_mapper(np.str('gs://ldeo-glaciology/REMA/processed/'+ice_shelf))


In [13]:
cluster.scale(8)

In [14]:
processed = xr.open_zarr(mapper)

In [15]:
if ice_shelf=='Baudouin': #Baudouin and Nivlisen are upside down
    processed = processed.apply(np.fliplr)
    processed = processed.apply(np.flipud)
    processed = processed.apply(np.fliplr)
    processed = processed.apply(np.flipud)
if ice_shelf=='Nivl': #Baudouin and Nivlisen are upside down
    processed = processed.apply(np.fliplr)
  #  processed = processed.apply(np.flipud)
  #  processed = processed.apply(np.fliplr)
  #  processed = processed.apply(np.flipud)
if ice_shelf=='Riiser': #Riiser is sideways
    processed = processed.transpose()
    
if ice_shelf=='LarsenC': #LarsenC is sideways
    processed = processed.transpose()

In [16]:
processed.attrs.update({'crs':'+init=epsg:3031'})
processed.attrs.update({'transform': (32.0, 0.0, 1600000.0, 0.0, -32.0, 800000.0)})
processed.attrs.update({'res': (32.0, 32.0)})
processed.attrs.update({'is_tiled': 1})
processed.attrs.update({'nodatavals': (-9999.0,)})
processed.attrs.update({'scales': (1.0,)})
processed.attrs.update({'offsets': (0.0,)})

In [17]:
processed.DB_filled.shape

(9217, 9773)

In [18]:
processed =  processed.chunk({'x': 2000,'y':2000})
cellsize = processed.res[1]

In [19]:
DB_label = processed.DB_unfilled.astype(int)

In [20]:
DB_label.shape

(9217, 9773)

In [ ]:
DB_label.coarsen(x=10,y=10,boundary='pad').mean().plot()

## Calculate the x,y coordinates of the chunks over which to calculate

In [ ]:
chunks_x = np.int_((np.cumsum(DB_label.chunks[0])))
chunks_y = np.int_((np.cumsum(DB_label.chunks[1])))
chunks_x = chunks_x[chunks_x<DB_label.shape[0]]
chunks_y = chunks_y[chunks_y<DB_label.shape[1]]

chunks_x = np.concatenate(([0],chunks_x,[DB_label.shape[0]-1]))
chunks_y = np.concatenate(([0],chunks_y,[DB_label.shape[1]-1]))
chunks_x, chunks_y

In [ ]:
chunks={}
counter = 0
for j in np.arange(1,len(chunks_y)):
    for i in np.arange(1,len(chunks_x)):
        chunks[counter] = [chunks_x[i-1],chunks_x[i], chunks_y[j-1],chunks_y[j]]
        counter = counter+1

In [ ]:
chunk_map = np.zeros(DB_label.shape)
for i in np.arange(0,len(chunks)):
    x1,x2,y1,y2 = chunks[i]
    chunk_map[x1:x2,y1:y2] = i

plt.imshow(chunk_map.transpose())
plt.colorbar()

In [ ]:
import os
import dask
from dask_gateway import Gateway
from dask.distributed import Client
import dask_gateway
import dask.array as da

In [ ]:
cluster.scale(10)

## If not first time: Import csv of drainage catchment properties

In [ ]:
cd 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import dask.dataframe
DB_merged_prop = pd.read_csv(str(ice_shelf+'_DB_merged_prop.csv'))

DB_merged_prop = DB_merged_prop.drop(columns='Unnamed: 0')

#coords_str = DB_merged_prop['coords']

#coords_list = [coords.strip('[]').split(',') for coords in coords_str]

coords_formatted = {}
for index,coord in DB_merged_prop.coords.iteritems():
    
    coords_formatted[index] = np.int_([x.strip('[ ]') for x in DB_merged_prop['coords'][index].strip('[ ]').split(',')])
    

DB_merged_prop['coords_reformatted'] = coords_formatted.values()

#del coords_str
#del coords_list
del coords_formatted

DB_merged_prop = dask.dataframe.from_pandas(DB_merged_prop,npartitions=10)

In [30]:
# Label lakes
islake = processed.P_all.chunk({'x':-1,'y':-1}).data>0

islake_filtered = ~dask.array.bitwise_not(islake).map_blocks(skimage.morphology.remove_small_holes, area_threshold=10)

islake_labelled_ = islake_filtered.map_blocks(skimage.measure.label,background=0)

islake_labelled = islake_labelled_.compute()

In [31]:
def lake_area(regionmask,intensity):
    return np.sum(intensity>0)/np.sum(regionmask)
def DB_regionprops2(DB,DB_filled):
    table_prop = skimage.measure.regionprops_table(DB,DB_filled,properties = ('label','max_intensity'),extra_properties=(lake_area,))
    #pd_prop = pd.DataFrame(table_prop,index=table_prop['label'])
    #pd.concat([b,pd_prop])
    return table_prop

In [32]:
b_dict = {'label','large_lake_drainage','lake_percent'}
label=[0]
DB_prop2 = pd.DataFrame(data={},index = label,columns=b_dict)
for x1,x2,y1,y2 in chunks.values():
    test_prop = dask.array.map_blocks(lambda DB,P_all: DB_regionprops2(DB,P_all),DB_label.data[x1:x2,y1:y2],islake_labelled[x1:x2,y1:y2], dtype=dict)
    test_prop = test_prop.compute()
    pd_prop = pd.DataFrame(data = {'label': test_prop['label'],'large_lake_drainage': test_prop['max_intensity'], 'lake_percent':test_prop['lake_area']},index=test_prop['label'])
    DB_prop2 = DB_prop2.append(pd_prop, ignore_index=True)

In [33]:
DB_prop2 = DB_prop2.groupby('label').agg({'large_lake_drainage':'first', 'lake_percent':'mean'}).reset_index()

In [34]:
DB_merged_prop = DB_merged_prop.merge(DB_prop2)

In [35]:
DB_merged_prop

,label,area,eccentricity,max_depth,catchment_volume,coords,Area_in_m,volume_to_area_ratio,coords_reformatted,large_lake_drainage,lake_percent
npartitions=10,,,,,,,,,,,
,int64,int64,float64,float64,float64,object,float64,float64,object,int64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [36]:
### Map of Ratio of Volume/Area of Catchments

### Calculate ratio of volume to area over each drainage area

In [37]:
large_lake_drainage_area = DB_merged_prop.groupby(DB_merged_prop.large_lake_drainage).Area_in_m.sum()
large_lake_drainage_area = large_lake_drainage_area.reset_index()


large_lake_drainage_volume = DB_merged_prop.groupby(DB_merged_prop.large_lake_drainage).catchment_volume.sum()

large_lake_drainage_volume = large_lake_drainage_volume.reset_index()


large_lake_drainage_maxdepth = DB_merged_prop.groupby(DB_merged_prop.large_lake_drainage).max_depth.max()

large_lake_drainage_maxdepth= large_lake_drainage_maxdepth.reset_index()



large_lake_drainage_lakepercent = DB_merged_prop.groupby(DB_merged_prop.large_lake_drainage).lake_percent.mean()

large_lake_drainage_lakepercent = large_lake_drainage_lakepercent.reset_index()

In [38]:
large_lake_drainage_area_coords = DB_merged_prop.groupby(DB_merged_prop.large_lake_drainage).coords_reformatted.aggregate([list]).reset_index()

In [39]:
cluster.scale(10)

In [42]:
import itertools
large_lake_drainage_area_coords_list = {}
for index, row in large_lake_drainage_area_coords.iterrows():
    large_lake_drainage_area_coords_list[row.large_lake_drainage] = list(itertools.chain(*row.list))

In [43]:
large_lake_drainage_area_coords = pd.Series(large_lake_drainage_area_coords_list,name='coordinates').sort_index()

In [44]:
Lake = large_lake_drainage_area.merge(large_lake_drainage_volume).merge(large_lake_drainage_maxdepth).merge(large_lake_drainage_lakepercent).set_index(large_lake_drainage_area.large_lake_drainage)

In [45]:
Lake = Lake.assign(ratio = lambda x: x['catchment_volume']/x['Area_in_m'])

 # Velocity

In [46]:
DB_prop4=pd.read_csv(ice_shelf+'_MEASUReS_velocity.csv')

In [47]:
DB_prop4 = DB_prop4.groupby('label').agg({'mean_velocity':'mean'}).reset_index()

In [48]:
DB_prop4 =  DB_prop4.sort_values('label').reset_index()

In [49]:
DB_merged_prop = DB_merged_prop.merge(DB_prop4)
DB_merged_prop


,label,area,eccentricity,max_depth,catchment_volume,coords,Area_in_m,volume_to_area_ratio,coords_reformatted,large_lake_drainage,lake_percent,index,mean_velocity
npartitions=10,,,,,,,,,,,,,
,int64,int64,float64,float64,float64,object,float64,float64,object,int64,float64,int64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [50]:
DB_prop4 = pd.read_csv(ice_shelf+'_MEASUReS_velocity.csv')

In [51]:
by_lake_mean_velocity = DB_merged_prop.groupby(DB_merged_prop.large_lake_drainage).mean_velocity.mean().reset_index().compute()
    
    
    


In [52]:
by_lake_mean_velocity = by_lake_mean_velocity.set_index(by_lake_mean_velocity.large_lake_drainage)

In [53]:
Lake = Lake.assign(velocity = by_lake_mean_velocity.mean_velocity)

# Subshelf Melt

In [54]:
ice_shelf

'LarsenC'

In [55]:
DB_prop5 = pd.read_csv(ice_shelf+'_subshelf_melt.csv')

In [56]:
DB_prop5 =  DB_prop5.groupby('label').subshelfmelt.sum().reset_index()


In [57]:
DB_prop5 = DB_prop5.set_index(DB_prop5.label)

In [58]:
DB_prop5['large_lake_drainage'] = DB_merged_prop.large_lake_drainage
DB_prop5

,label,subshelfmelt,large_lake_drainage
label,,,
1.0,1.0,0.0,0
3.0,3.0,0.0,49106
4.0,4.0,0.0,49106
5.0,5.0,0.0,48810
6.0,6.0,0.0,49106
...,...,...,...
277237.0,277237.0,0.0,0
277238.0,277238.0,0.0,0
277239.0,277239.0,0.0,0


In [59]:
by_lake_subshelf_melt = DB_prop5.groupby(DB_prop5.large_lake_drainage).subshelfmelt.sum().reset_index()
    
    


In [60]:
by_lake_subshelf_melt = by_lake_subshelf_melt.set_index(by_lake_subshelf_melt.large_lake_drainage)

In [61]:
Lake = Lake.assign(subshelf_melt = by_lake_subshelf_melt.subshelfmelt)

In [62]:
Lake

,large_lake_drainage,Area_in_m,catchment_volume,max_depth,lake_percent,ratio,velocity,subshelf_melt
npartitions=1,,,,,,,,
0,int64,float64,float64,float64,float64,float64,float64,float64
63083,...,...,...,...,...,...,...,...


# Vulnerability

In [63]:
DB_prop6 = pd.read_csv(ice_shelf+'_ice_shelf_vulnerability.csv')

In [64]:
DB_prop6 =  DB_prop6.groupby('label').agg({'vulnerability':'max'}).reset_index()


In [65]:
DB_prop6 = DB_prop6.assign(is_floating = np.greater_equal(DB_prop6.vulnerability,0))

In [66]:
DB_prop6 = DB_prop6.set_index(DB_prop6.label)

In [67]:
DB_prop6 = DB_prop6.assign(large_lake_drainage = DB_merged_prop.large_lake_drainage)

In [68]:
by_lake_vulnerability = DB_prop6.groupby(DB_prop6.large_lake_drainage).vulnerability.mean().reset_index()
    
    


In [69]:
by_lake_vulnerability = by_lake_vulnerability.set_index(by_lake_vulnerability.large_lake_drainage)

In [70]:
Lake = Lake.assign(vulnerability = by_lake_vulnerability.vulnerability)

In [71]:
Lake

,large_lake_drainage,Area_in_m,catchment_volume,max_depth,lake_percent,ratio,velocity,subshelf_melt,vulnerability
npartitions=1,,,,,,,,,
0,int64,float64,float64,float64,float64,float64,float64,float64,float64
63083,...,...,...,...,...,...,...,...,...


# SM

In [72]:
DB_prop8 = pd.read_csv(ice_shelf+'_RACMO_SM.csv')

In [73]:
DB_prop8 = DB_prop8.groupby('label').agg({'mean_SM':'mean',
                                         'area':'sum'}).reset_index()

In [74]:
DB_prop8 = DB_prop8.assign(annual_SM_in_m= DB_prop8.mean_SM)#(32**2)*DB_prop8.area*

In [75]:
DB_prop8 = DB_prop8.assign(annual_SM_in_m_volume= DB_prop8.annual_SM_in_m*(32**2)*DB_prop8.area)

In [76]:
DB_prop8 = DB_prop8.set_index(DB_prop8.label)

In [77]:
DB_prop8 = DB_prop8.assign(large_lake_drainage = DB_merged_prop.large_lake_drainage)

In [78]:
by_lake_SM = DB_prop8.groupby(DB_prop8.large_lake_drainage).agg({'annual_SM_in_m_volume':'sum','area':'sum'}).reset_index()
    
    


In [79]:
by_lake_SM = by_lake_SM.set_index(by_lake_SM.large_lake_drainage)

In [80]:
by_lake_SM

,large_lake_drainage,annual_SM_in_m_volume,area
large_lake_drainage,,,
0,0,1.654500e+09,12582234
1,1,2.198093e+05,687
2,2,2.428461e+05,759
3,3,1.838491e+05,561
4,4,5.715381e+05,1744
...,...,...,...
63079,63079,2.992783e+00,84
63080,63080,7.375074e+00,207
63081,63081,1.726495e+04,100


In [81]:
Lake = Lake.assign(SM = by_lake_SM.annual_SM_in_m_volume/(by_lake_SM.area*32**2))

In [82]:
Lake.compute().to_csv(ice_shelf+'_props_by_lake.csv')

In [83]:
cluster.shutdown()

In [88]:
Lake = pd.read_csv(ice_shelf+'_props_by_lake.csv')

In [101]:
output_map = np.zeros(DB_label.shape)
for index,row in Lake.iterrows():
    [x,y] = np.unravel_index(large_lake_drainage_area_coords[index], DB_label.shape)
    output_map[x,y] = row.SM

if np.isin(ice_shelf,['Baudouin','Nivl']): #Baudouin is upside down
    output_map = np.fliplr(output_map)
    output_map = np.flipud(output_map)
    output_map = np.fliplr(output_map)
    output_map = np.flipud(output_map)

KeyError: 146

In [ ]:
plt.imshow(output_map)
plt.colorbar()

In [ ]:
cd

In [ ]:
large_lake_drainage = processed.P_all

large_lake_drainage.data = output_map;
large_lake_drainage.name = 'annual_average_surface_melt'

In [ ]:
large_lake_drainage.to_netcdf(ice_shelf+'_surface_melt_by_lake_m.nc')

## Comparing to Future Melt

In [102]:
cd

/home/jovyan


In [103]:
cluster = gateway.new_cluster()

In [104]:
client = Client(cluster)

In [105]:
cluster.scale(12)

### Decadal Avg

In [2]:
models = ['ACCESS1.3','CESM2','CNRM-CM6','NorESM']

In [3]:
CMIP = xr.open_dataset('Kittel_futureSMB/year-MAR_NorESM-1980-2100_zen.nc2',)
rhow = 1000
CMIP_RU = CMIP.RU.where(xr.ufuncs.isfinite(CMIP.RU),-9999)/rhow

NameError: name 'xr' is not defined

In [108]:
CMIP_RU =CMIP_RU.assign_coords({'X':CMIP_RU.X*1000, 'Y':CMIP_RU.Y*1000}).rio.write_crs('epsg:3031')

CMIP_RU = CMIP_RU.rio.set_spatial_dims('X','Y').rename({'X':'x','Y':'y'})

CMIP_RU_decadal = CMIP_RU.groupby(np.floor(CMIP_RU.TIME.dt.year/5)).max().squeeze()

CMIP_RU_decadal = CMIP_RU_decadal[9:,:,:] #After 2020

In [ ]:
ratio_map = xr.open_dataarray(ice_shelf + '_all_lake_drainage_area_to_volume.nc')

x1,y1,x2,y2 = processed.rio.bounds()
maxx = max(x1,x2)
minx = min(x1,x2)
maxy = max(y1, y2)
miny = min(y1,y2)
if ice_shelf=='Baudouin' or ice_shelf=='Nivl': 
    CMIP_RU_decadal_clipped = CMIP_RU_decadal.rio.clip_box(minx,miny,maxx,maxy)
else:
    CMIP_RU_decadal_clipped = CMIP_RU_decadal.rio.clip_box(miny,minx,maxy,maxx)

Y = processed.y.values
X = processed.x.values
mesh_X, mesh_Y = np.meshgrid(X,Y)
#if name=='Baudouin' or name=='Nivl': 
 #   coords = [mesh_X.ravel(),mesh_Y.ravel()]
#else:
coords = [mesh_Y.ravel(),mesh_X.ravel()]

CMIP_X,CMIP_Y = np.meshgrid(CMIP_RU_decadal_clipped.x, CMIP_RU_decadal_clipped.y)

points = np.transpose([CMIP_Y.ravel(), CMIP_X.ravel()])

xi = np.transpose(coords)
years = CMIP_RU_decadal_clipped.year.values*5
whenlarger = np.zeros(islake_labelled.data.shape)

for t in np.arange(0, len(CMIP_RU_decadal_clipped.year)):
    #if name=='Baudouin' or name=='Nivl':
     #   data = CESM_RU_decadal_clipped[t,:,:].transpose().data.ravel()
    #else:
    data = CMIP_RU_decadal_clipped[t,:,:].data.transpose().ravel()

    CMIP_interp = scipy.interpolate.griddata(points,data,xi,method='nearest')

    CMIP_interp = CMIP_interp.reshape([ratio_map.shape[0],ratio_map.shape[1]])

    mask = whenlarger==0

    whenlarger[np.bitwise_and(CMIP_interp>ratio_map, mask)] = years[t]
    
    

In [1]:
plt.imshow(whenlarger)
plt.colorbar()

NameError: name 'plt' is not defined

In [ ]:
plt.imshow(CMIP_RU_decadal_clipped.sum(dim='year').transpose(),vmin=0, vmax=1)
plt.colorbar()

In [ ]:
ratio_map[5:,5:].coarsen(x=10,y=10).mean().transpose().plot(vmin=0, vmax=1)

In [ ]:
fig = plt.figure(figsize=(20,20))
counter = 1
for index, row in IS_processed.iterrows():

    if row.NAME=='Riiser-Larsen':
        name = 'Riiser'
    else:
        name = row.NAME

    ratio_map = xr.open_dataarray(name + '_all_lake_drainage_area_to_volume.nc')

    x1,y1,x2,y2 = ratio_map.rio.bounds()
    maxx = max(x1,x2)
    minx = min(x1,x2)
    maxy = max(y1, y2)
    miny = min(y1,y2)
    if name=='Baudouin' or name=='Nivl': 
        CESM_RU_decadal_clipped = CESM_RU_decadal.rio.clip_box(minx,miny,maxx,maxy)
    else:
        CESM_RU_decadal_clipped = CESM_RU_decadal.rio.clip_box(miny,minx,maxy,maxx)

    Y = ratio_map.y.values
    X = ratio_map.x.values
    mesh_X, mesh_Y = np.meshgrid(X,Y)
    if name=='Baudouin' or name=='Nivl': 
        coords = [mesh_X.ravel(),mesh_Y.ravel()]
    else:
        coords = [mesh_Y.ravel(),mesh_X.ravel()]


    CESM_X,CESM_Y = np.meshgrid(CESM_RU_decadal_clipped.x, CESM_RU_decadal_clipped.y)

    points = np.transpose([CESM_X.ravel(), CESM_Y.ravel()])

    xi = np.transpose(coords)
    years = CESM_RU_decadal_clipped.year.values
    whenlarger = np.zeros(ratio_map.data.shape)
    for t in np.arange(0, len(CESM_RU_decadal_clipped.year)):
        if name=='Baudouin' or name=='Nivl':
            data = CESM_RU_decadal_clipped[t,:,:].transpose().data.ravel()
        else:
            data = CESM_RU_decadal_clipped[t,:,:].data.ravel()

        CMIP_interp = scipy.interpolate.griddata(points,data,xi,method='nearest')

        CMIP_interp = CMIP_interp.reshape([ratio_map.shape[0],ratio_map.shape[1]])
        
        mask = whenlarger==0
        
        whenlarger[np.bitwise_and(CMIP_interp>ratio_map, mask)] = years[t]
        

    #decade_list = CESM_RU_decadal_clipped.year.values*10
    #year_list = CESM_RU_accum.TIME.dt.year

    #larger_decade=np.zeros(ratio_interp_ravelled.shape)
    #for x in np.arange(0, len(ratio_interp_ravelled)):
     #   if ratio_interp_ravelled[x]!=0:
      #      if np.any(ratio_interp_ravelled[x]<CESM_ravelled[x,:]):      
       #         larger_decade[x] = decade_list[np.argwhere(ratio_interp_ravelled[x]<CESM_ravelled[x,:])[0]]
        #    else:
         #       larger_decade[x] = np.nan
        #else:
         #   larger_decade[x] = 0

    #larger_decade = larger_decade.reshape([CESM_RU_decadal_clipped.shape[1],CESM_RU_decadal_clipped.shape[2]])
   
    ax = fig.add_subplot(3,2, counter)
    ax.set_title(name)
    cmap = mpl.cm.inferno
    #cmap.set_bad('black',1.)
    #larger_decade = np.ma.array(larger_decade, mask=np.isnan(larger_decade))
    if name=='Baudouin' or name=='Nivl':
        h=ax.pcolormesh(Y,X,whenlarger, shading='auto',vmin=2010,vmax=2100, cmap=cmap)
    else:
        h=ax.pcolormesh(X,Y,whenlarger, shading='auto',vmin=2010,vmax=2100, cmap=cmap)



    plt.colorbar(h)
    IS_processed[IS_processed.NAME==row.NAME].geometry.plot(ax=ax,facecolor='None', edgecolor='b')
    counter = counter+1

### Accumulative

In [ ]:
cd

In [ ]:
CNRM = xr.open_dataset('Kittel_futureSMB/year-MAR_NorESM-1980-2100_zen.nc2')
rhow = 1000
CNRM_RU = CNRM.RU.where(xr.ufuncs.isfinite(CNRM.RU),-9999)/rhow

In [ ]:
CNRM_RU =CNRM_RU.assign_coords({'X':CNRM_RU.X*1000, 'Y':CNRM_RU.Y*1000}).rio.write_crs('epsg:3031')

CNRM_RU = CNRM_RU.rio.set_spatial_dims('X','Y').rename({'X':'x','Y':'y'})

CNRM_RU_decadal = CNRM_RU.groupby(np.floor(CNRM_RU.TIME.dt.year/10)).mean().squeeze()

CNRM_RU_decadal = CNRM_RU_decadal[4:,:,:]

In [ ]:
CNRM_RU_accum = CNRM_RU[CNRM_RU.TIME.dt.year>=2015,0,:,:].cumsum(dim='TIME')

In [ ]:
fig = plt.figure(figsize=(20,20))
counter = 1
for index, row in IS_processed.iterrows():

    if row.NAME=='Riiser-Larsen':
        name = 'Riiser'
    else:
        name = row.NAME

    ratio_map = xr.open_dataarray(name + '_all_lake_drainage_area_to_volume.nc')

    x1,y1,x2,y2 = ratio_map.rio.bounds()
    maxx = max(x1,x2)
    minx = min(x1,x2)
    maxy = max(y1, y2)
    miny = min(y1,y2)
    if name=='Baudouin' or name=='Nivl': 
        CNRM_RU_accum_clipped = CNRM_RU_accum.rio.clip_box(minx,miny,maxx,maxy)
    else:
        CNRM_RU_accum_clipped = CNRM_RU_accum.rio.clip_box(miny,minx,maxy,maxx)

    Y = ratio_map.y.values
    X = ratio_map.x.values
    mesh_X, mesh_Y = np.meshgrid(X,Y)
    if name=='Baudouin' or name=='Nivl': 
        coords = [mesh_X.ravel(),mesh_Y.ravel()]
    else:
        coords = [mesh_Y.ravel(),mesh_X.ravel()]


    CNRM_X,CNRM_Y = np.meshgrid(CNRM_RU_accum_clipped.x, CNRM_RU_accum_clipped.y)

    xi = np.transpose([CNRM_X.ravel(), CNRM_Y.ravel()])

    points = np.transpose(coords)
    if name=='Baudouin' or name=='Nivl':
        data = ratio_map.transpose().data.ravel()
    else:
        data = ratio_map.data.ravel()

    ratio_interp = scipy.interpolate.griddata(points,data,xi,method='nearest')

    ratio_interp = ratio_interp.reshape([CNRM_RU_accum_clipped.shape[1],CNRM_RU_accum_clipped.shape[2]])

    ratio_interp_xr = xr.DataArray(ratio_interp, (CNRM_RU_accum_clipped.y,CNRM_RU_accum_clipped.x))
    CNRM_ravelled = CNRM_RU_accum_clipped.data.reshape([CNRM_RU_accum_clipped.shape[1]* CNRM_RU_accum_clipped.shape[2],CNRM_RU_accum_clipped.shape[0]])
    ratio_interp_ravelled = ratio_interp_xr.data.ravel()

    #decade_list = CESM_RU_decadal_clipped.year.values*10
    year_list = CNRM_RU_accum.TIME.dt.year

    larger_decade=np.zeros(ratio_interp_ravelled.shape)
    for x in np.arange(0, len(ratio_interp_ravelled)):
        if ratio_interp_ravelled[x]!=0:
            if np.any(ratio_interp_ravelled[x]<CNRM_ravelled[x,:]):      
                larger_decade[x] = year_list[np.argwhere(ratio_interp_ravelled[x]<CNRM_ravelled[x,:])[0]]
            else:
                larger_decade[x] = np.nan
        else:
            larger_decade[x] = 0

    larger_decade = larger_decade.reshape([CNRM_RU_accum_clipped.shape[1],CNRM_RU_accum_clipped.shape[2]])
   
    ax = fig.add_subplot(3,2, counter)
    ax.set_title(name)
    cmap = mpl.cm.tab20b
    cmap.set_bad('black',1.)
    larger_decade = np.ma.array(larger_decade, mask=np.isnan(larger_decade))

    h=ax.pcolormesh(CNRM_RU_accum_clipped.x,CNRM_RU_accum_clipped.y,larger_decade, shading='auto',vmin=2000,vmax=2100, cmap=cmap)


    plt.colorbar(h)
    IS_processed[IS_processed.NAME==row.NAME].geometry.plot(ax=ax,facecolor='None', edgecolor='b')
    counter = counter+1

In [ ]:
cluster.shutdown()